# Get a Constellation graph
Get some vertex and transaction attributes.

In [ ]:
import json
from PIL import Image
import io
import re

import constellation_client

In [ ]:
SPRITE_ATLAS = 'sprite_atlas.png'

# Constellation icons are this big.
#
ICON_SIZE = 256

# We build a texture map this big.
# Even low-end GPUs should be able to cope with this.
#
TEXTURE_SIZE = 4096

TEXTURE_ICONS = TEXTURE_SIZE // ICON_SIZE

print(f'Maximum number of icons: {TEXTURE_ICONS**2}')

In [ ]:
cc = constellation_client.Constellation()

In [ ]:
# named_colors = cc.call_service('list_named_colors').json()

# for name, html in named_colors.items():
#     gl = htmlToGl_4(html)
#     print(name, html, gl)

In [ ]:
def colorToFloat4(c):
    """Convert a color name or an HTML #rrggbb color or a str(list of floats) to a GL list of floats between 0.0 and 1.0 color, and alpha."""
    
    def get_color_names():
        if not hasattr(colorToFloat4, 'named_colors'):
            colorToFloat4.named_colors = cc.call_service('list_named_colors').json()
        
        return colorToFloat4.named_colors
        
    if not c:
        raise ValueError(f'Bad color "{c}"')
    
    if c[0]=='[' and c[-1]==']':
        color = c[1:-1].split(',')
        color = [float(i) for i in color]
        if len(color)==3:
            color.append(1.0)
        
        return color
        
    if len(c)!=7 or not c.startswith('#'):
        cn = get_color_names()
        if c not in cn:
            raise ValueError(f'Bad HTML color "{c}"')
        
        c = cn[c]
    
    r = int(c[1:3], 16)
    g = int(c[3:5], 16)
    b = int(c[5:7], 16)
    
    return [r/255, g/255, b/255, 1.0]

In [ ]:
def parse_blaze(b):
    """Extract the color and angle from the blaze string."""
    
    if not b:
        return None

    angle, color = b.split(';')
    angle = float(angle)
    color = colorToFloat4(color)
    
    return {'angle':angle, 'color':color}

## Graph

In [ ]:
gdf = cc.get_graph_attributes()
print(gdf.loc[0, 'camera'])
gdf.to_dict(orient='records')

In [ ]:
def get_camera(s):
    """Get the camera eye,centre,up from the camera string."""
    
    pos = re.compile(r'(\w+): 3f\[(.+?)\]')
    camera = {}
    for name,loc in pos.findall(s):
        camera[name] = [float(f) for f in loc.split(',')]

    return camera

camera = get_camera(gdf.loc[0, 'camera'])
camera

In [ ]:
background_color = gdf.loc[0, 'background_color']
node_color_col = gdf.loc[0, 'node_color_reference']
if node_color_col is None:
    node_color_col = 'color'

## Vertices

In [ ]:
vx_attrs = [
    'source.[id]',
    'source.Label',
    'source.x',
    'source.y',
    'source.z',
    'source.icon',
    'source.background_icon',
    'source.nradius',
    'source.blaze',
    f'source.{node_color_col}' # TODO do this for txs as well
]

vxdf = cc.get_dataframe(vx=True, attrs=vx_attrs)
vxdf = vxdf.rename(columns=lambda name:name.replace('source.', '', 1) if name.startswith('source.') else name)

if node_color_col:
    vxdf['color'] = vxdf[node_color_col]
    
vxdf.blaze = vxdf.blaze.apply(parse_blaze)

vxdf.head()

## Icons

In [ ]:
def make_texture_atlas(icons, atlas_name):
    """Given a collection of icons of size ICON_SIZE, create a texture atlas.
    
    TODO put the background icons in first?
    
    :returns: the width,height of the atlas, a map from name to cell_index
    """
    
    icons = set(icons)
    
    N = len(icons)
    W = min(N, TEXTURE_ICONS)
    H = N % W + 1
    if H >= TEXTURE_ICONS:
        raise ValueError('Too many icons')
    
    print(N, W, H)
    
    sheet = Image.new('RGBA', (W*ICON_SIZE, H*ICON_SIZE))
    print(f'Created texture atlas {sheet.size}')
    
    icon_map = {}
    for i, icon_name in enumerate(icons):
        x = i % W
        y = i // W
        cell_index = y*W + x
        print('.', x, y, cell_index, icon_name)
        
        img = cc.get_icon(icon_name)
        with io.BytesIO(img) as buf:
            img = Image.open(buf)
            
            ISIZE = ICON_SIZE, ICON_SIZE
            if img.size>ISIZE:
                img = img.thumbnail(ISIZE)
            
            if img.size[0]<ICON_SIZE or img.size[1]<ICON_SIZE:
                img_size = Image.new('RGBA', ISIZE, (255, 255, 255, 0))
                dx = (ICON_SIZE-img.size[0])//2
                dy = (ICON_SIZE-img.size[1])//2
                img_size.paste(img, (dx, dy))
                img = img_size
        
            print('  ', (x*ICON_SIZE, y*ICON_SIZE))
            sheet.paste(img, (x*ICON_SIZE, y*ICON_SIZE))
            icon_map[icon_name] = y*N + i
    
    print('Save texture atlas to {atlas_name}')
    sheet.save(atlas_name)
    
    return W, H, icon_map

In [ ]:
icons = set(vxdf['icon']) | set(vxdf['background_icon'])
width, height, icon_map = make_texture_atlas(icons, SPRITE_ATLAS)
vxdf['fg_icon_index'] = vxdf.icon.apply(lambda icon_name:icon_map[icon_name] if icon_name else 0)
vxdf['bg_icon_index'] = vxdf.background_icon.apply(lambda icon_name:icon_map[icon_name] if icon_name else 0)

In [ ]:
vxs = {}
for vx in vxdf.to_dict(orient='records'):
    vx_id = vx.pop('[id]')
#     print(vx)
    vxs[vx_id] = vx
# vxs

## Transactions

In [ ]:
mix_color = gdf.loc[0, 'mix_color']

tx_attrs = ['source.[id]', 'destination.[id]', 'transaction.color', 'transaction.directed']

txdf = cc.get_dataframe(tx=True, attrs=tx_attrs)
txdf = txdf.rename(columns=lambda name:name.replace('transaction.', '', 1) if name.startswith('transaction.') else name)
txdf = txdf.rename(columns={'source.[id]':'sid_', 'destination.[id]':'did_'})

print(txdf.shape)
txdf.head(10)

In [ ]:
def line_directions(df, lines):
    """What directions are the transactions in the given rows?

    Return a string containing one or both of '>' (directed) or '|' (undirected).
    """

    directed = set(df.loc[lines, 'directed'])
    is_dir = '>' if True in directed else ''
    is_not_dir = '|' if False in directed else ''
    
    return is_dir + is_not_dir

def line_color(df, lines):
    """Get the (possibly mixed) color of the given rows."""

    colors = list(df.loc[lines, 'color'])
    n_colors = len(set(tuple(c) for c in colors))
    if n_colors==1:
        return colors[0]
    else:
        return mix_color

def get_lines(txdf):
    """Get the direction(s) and color of the combined transactions from src to dst.
    
    If there are transactions from dst to src, get the direction(s) and color of those,
    and combine them with the src->dst direction(s) and color, because the front-end
    only draws one line.
    """
    
    grp = txdf.groupby(['sid_', 'did_'])
    items = dict(grp.indices.items())
    txs = []
    while items:
        (src,dst), tx_ixs = items.popitem()
        dirs = line_directions(txdf, tx_ixs)
        color = line_color(txdf, tx_ixs)
        if (dst,src) in items:
            tx_ixs_alt = items.pop((dst, src))
            dirs_alt = line_directions(txdf, tx_ixs_alt)
            color_alt = line_color(txdf, tx_ixs_alt)

            if '>' in dirs_alt:
                dirs += '<'
            if '|' in dirs_alt and '|' not in dirs:
                lines += '|'

            if color_alt!=color:
                color = mix_color
        
        txs.append({
            'sid_': src,
            'did_': dst,
            'color': color,
            'directions': dirs
        })
    
    return txs

txs = get_lines(txdf)
txs

In [ ]:
# mix_color = gdf.loc[0, 'mix_color']

# # TODO treat multiple transactions correctly.
# #
# txdf = txdf.drop_duplicates(['source.[id]', 'destination.[id]'])
# print(txdf.shape)

In [ ]:
# txs = txdf.to_dict(orient='records')
# txs

In [ ]:
# Write the JSON document for the Constellation web viewer.
#
with open('sphere-graph.json', 'w') as f:
    json.dump({
        'vertex': vxs,
        'transaction': txs,
        'sprite_atlas': {
            'name': SPRITE_ATLAS,
            'width': width,
            'height': height
            },
        'icon': icon_map,
        'camera': camera,
        'background_color': background_color
        }, f, indent=2)

# An example graph

In [ ]:
import networkx as nx
import pandas as pd

In [ ]:
g = nx.les_miserables_graph()
people = [[p1, p2] for p1,p2 in g.edges]
df = pd.DataFrame.from_records(people)
df.columns = ['source.Identifier', 'destination.Identifier']
df['source.Type'] = 'Person'
df['destination.Type'] = 'Person'
df['transaction.Type'] = 'Relationship'
df.head()

In [ ]:
cc = constellation_client.Constellation()
cc.put_dataframe(df)